See the final report appendix or the final project data assignment for more information on the data generation for this project

In [1]:
using Surrogates,Flux,Statistics,DifferentialEquations,Plots,Suppressor,MLDataUtils,DelimitedFiles
using Flux.Data: DataLoader
using DelimitedFiles
include("soluteperm.jl");
include("utilityfunc.jl");

In [2]:
#set lower and upper bounds on Lpt and Kt (Lpt is first element and Kt is second element)
lb=Float32[5e-7,5e-7,5]
ub=Float32[5e-6,5e-6,30]

3-element Vector{Float32}:
  5.0f-6
  5.0f-6
 30.0

In [3]:
conc_ub=.6
i_lb=Float32[5e-7,7e-7,5.0,0.0,0.0]
i_ub=Float32[3.5e-6,4e-6,30,(300/3600),conc_ub]
iscaler(x)=(x-i_lb)./(i_ub-i_lb)
inv_iscaler(x)=x.*(i_ub.-i_lb).+i_lb
oscaler(x)= x ./ conc_ub
inv_oscaler(x)=x.*conc_ub

inv_oscaler (generic function with 1 method)

In [4]:
function iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=iscaler(x[:,i])
    end
    return out
end

iscalerbatch (generic function with 1 method)

In [5]:
function inv_iscalerbatch(x)
    nfeat,sl = size(x)
    out=Array{Float32,2}(undef,nfeat,sl)
    for i=1:sl
        out[:,i]=inv_iscaler(x[:,i])
    end
    return out
end

inv_iscalerbatch (generic function with 1 method)

In [6]:
function Diffusion(rs)
    Diam_parc = 2*rs
    return D = 1.981e-06*Diam_parc^(-1.157) + 2.221e-08
end

function Blood_half_life(rs)
    Diam_parc = 2*rs
    a1 = 1081
    b1 = -16.63
    c1 = 84.82
    a2 = 517.4
    b2 = 65.61
    c2 = 996.6
    return kd = (a1*exp(-((Diam_parc-b1)/c1)^2) + a2*exp(-((Diam_parc-b2)/c2)^2))*60
end
function a_pressure(r,α)
    return r == 0 ? 1 - csch(α) : 1 - sinh(α*r)/(r*sinh(α))
    #return 1 - sinh(α*r)/(r*sinh(α))
end
function a_pressure_prime(r,α)
    return r == 0 ? 0 : (sinh(α*r)-α*r*cosh(r*α))/(sinh(α)*r^2)
    #return (sinh(α*r)-α*r*cosh(r*α))/(sinh(α)*r^2)
end
function MSTHighPeclet!(dc,c,p,t)
    #dc = preallocated vector corresponding to f(c,t)= dC/dt
    #c = concentration vector at all spatial points
    #p = vector of parameters as define in modelanalysis
    #t = current time
    #Unpackage parameters
    P,N,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd=p
    co=1 #initial concentration
    tspan=3600 #timespan
    cv=co*exp(-t*tspan/kd) #time-dependent exponential decay term
    Pe=Lpt*Pv*(Pvv-P[1])*(1-sigma)/Perm #Initial Peclet value
    #interior boundary condition
    dc[1]=tspan*(2*D*(c[2]-c[1])/dr^2 +Lpt*Svt*Pv*(Pvv-P[1])*(1-sigma)*cv)
    for j=2:N-1
        dc[j]=tspan*(((2*D/r[j])*((c[j+1]-c[j])/dr))+(D*(c[j+1]-2*c[j]+c[j-1])/dr^2)+ Kt*((P[j+1]-P[j-1])/(2*dr))*((c[j+1]-c[j])/dr)+ Lpt*Svt*Pv*(Pvv-P[j])*(1-sigma)*cv)
    end
    dc[N]=0 # concentration at the outtermost boundary
end
function MST!(dc,c,p,t)
    #dc = preallocated vector corresponding to f(c,t)= dC/dt
    #c = concentration vector at all spatial points
    #p = vector of parameters as define in modelanalysis
    #t = current time
    #Unpackage parameters
    P,N,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd=p
    co=1 #initial concentration
    tspan=3600 #timespan
    cv=co*exp(-t*tspan/kd) #time-dependent exponential decay term
    Pe=Lpt*Pv*(Pvv-P[1])*(1-sigma)/Perm #Initial Peclet value
    dc[1]= tspan*(2*D*(c[2]-c[1])*dr^-2 + Lpt*Svt*Pv*(Pvv-P[1])*(1-sigma)*(cv*exp(Pe)-c[1])/(exp(Pe)-1)) # R=0 boundary condition
    for j=2:N-1
        #Define interior nodes
        Pe=Lpt*Pv*(Pvv-P[j])*(1-sigma)/Perm #Peclet number
        dc[j]=tspan*(((2*D/r[j])*((c[j+1]-c[j])/dr))+(D*(c[j+1]-2*c[j]+c[j-1])/dr^2)+ Kt*((P[j+1]-P[j-1])/(2*dr))*((c[j+1]-c[j])/dr)+ Lpt*Svt*Pv*(Pvv-P[j])*(1-sigma)*(cv*exp(Pe)-c[j])/(exp(Pe)-1))
    end
    dc[N]=0 # concentration at the outtermost boundary
end
function Accumulation_Model(sol,n_spatial,n_time,n_nodes)
    c_model=zeros(n_nodes)
    spacingfactor = n_time ÷ n_nodes
    for j=1:n_nodes
        c_model[j]=mean(sol[:,spacingfactor*j])
    end
    return c_model
end
function SepBatch(Lpt,Kt,rs,t,accum_dat,n)
    idat = zeros(5,n-1)
    odat=zeros(n-1)
    for i=1:n-1
        idat[:,i]=[Lpt,Kt,rs,t[i],accum_dat[i]]
        odat[i]=accum_dat[i+1]
    end
    return idat,odat
end
function accumlation_intermediate(x) #x is a vector containing Lpt and Kt
    n_spatial=100
    n_time=21
    n_nodes = 21 # number of nodes desired to be outputted for accumulation data
    Lpt,Kt,rs = x
    Svt = 200;      #tumor vascular density
    D = Diffusion(rs)
    kd = Blood_half_life(rs)
    Perm,sigma=soluteperm(Lpt,rs) #Vascular permeability and solute reflection coefficient respectively
    R=1. # Tumor Radius
    Pv=25. #
    Pvv=1.
    att=R*sqrt(Lpt*Svt/Kt) #parameter alpha for tumor, ignoring lymphatics
    Press(r)= a_pressure(r,att)
    co=1.
    tdomain=300

    r= (1/R)*(range(0,stop=R,length=n_spatial))
    dr=1/(n_spatial-1)

    P= broadcast(Press,r)

    c0=zeros(n_spatial,1)
    c0[end]=0.
    #define timespan
    time_end =(300/3600)
    tspan=(0.,time_end)
    #package parameters
    p=P,n_spatial,sigma,Perm,Lpt,Svt,Kt,Pvv,Pv,D,r,dr,kd
    #Define and solve system of ODEs using time constant of 1/n_nodes and the QNDF solver for stiff systems
    dt=time_end/(n_time-1)
    t = dt*[i for i=0:n_nodes-1]
    @suppress_err begin
        prob=ODEProblem(MST!,c0,tspan,p)
        sol= solve(prob,QNDF(),saveat=dt)
        if length(sol.t)==1
            prob=ODEProblem(MSTHighPeclet!,c0,tspan,p)
            sol= solve(prob,QNDF(),saveat=dt)
        end
        accum_dat = Accumulation_Model(sol,n_spatial,n_time,n_nodes)
        return SepBatch(Lpt,Kt,rs,t,accum_dat,n_nodes)
    end
end
function prepdata(s)
   x,y= accumlation_intermediate(s)
    return Float32.(iscalerbatch(x)),reshape(Float32.(oscaler.(y)),(1,20))
end
#accum(x) =  output_scaler(accumlation_intermediate(invscaler(x)))

prepdata (generic function with 1 method)

In [7]:
n_samp=Int64(1e4); # number of samples
nbatch = n_samp; # number of batches
nfeat=5; #5 features
ntarg=1; #1 target values
sl=20; #sequence length of 20

In [8]:
#Create Data Sets
#First Randomly Sample the biological parameters using a Sobol Sequence
S=Float32.(reshape([x[j] for x in (sample(n_samp, lb,ub, SobolSample())) for j in eachindex(x)],(3,n_samp)));
#Define empty arrays to store inputs and outputs into
X = Array{Float32, 2}(undef,5, n_samp*20);
Y= Array{Float32, 2}(undef,1, n_samp*20);
#Fill in the arrays using the numerical model
for i=1:n_samp
    X[:,(20*(i-1)+1):(20*i)],Y[1,(20*(i-1)+1):(20*i)]= accumlation_intermediate(S[:,i][:])
end

In [9]:
data = [X ; Y]

6×200000 Matrix{Float32}:
  5.00824f-7   5.00824f-7   5.00824f-7  …  8.01437f-7  8.01437f-7  8.01437f-7
  1.69998f-6   1.69998f-6   1.69998f-6     4.0468f-6   4.0468f-6   4.0468f-6
 11.3431      11.3431      11.3431         7.64359     7.64359     7.64359
  0.0          0.00416667   0.00833333     0.0708333   0.075       0.0791667
  0.0          0.00512395   0.0102461      0.191997    0.203264    0.214528
  0.00512395   0.0102461    0.0153663   …  0.203264    0.214528    0.225788

In [10]:
#writedlm( "MLFinalData_v.csv",  data, ',')